In [525]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

import json


#spark session
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F


import os
import re 

#folium
import geopandas as gpd
import re
import folium

In [526]:
# Create a spark session

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

#read in the parquet
domain = spark.read.parquet('../data/raw/domain-website-data')


In [527]:
#drop NSW
domain = domain.filter(F.col('state') == 'VIC')

#drop column with too many missing values
domain = domain.drop('land_size','land_unit','is_rural','is_retirement','url').toPandas()

#filter property type, remove 'cat spcae' and 'Acreage / Semi-Rural'
selected_property_type = ['Apartment / Unit / Flat', 'Studio', 'Townhouse',
       'House', 'New House & Land', 'Semi-Detached', 'Villa', 'Terrace',
       'Acreage / Semi-Rural', 'New Apartments / Off the Plan', 'Duplex',
       'Unknown', 'Farm', 'Penthouse', 'Rural']

domain= domain.loc[domain['property_type'].isin(selected_property_type)]

In [528]:
#remove instance with 'carpark' in street name
def carpark_flag(s):
    pattern = re.compile('carpark|CP|car park|carspace')
    match = re.match(pattern,s)
    #o for non carpark, 1 for carpark
    if match is None:
        return 0
    else:
        return 1

In [529]:
def get_rent(s):
    pattern = re.compile('^( )*([0-9]*)( )*')
    num = re.sub('(,)?', '',s)
    num = re.sub('[^0-9]+', ' ',num)
    match = re.match(pattern,num)[0].replace(" ", "")

    if match !='':
        return match
    else:
        return 0

In [534]:
domain['street'] = domain['street'].str.lower()
domain['carpark_flag'] = [carpark_flag(i) for i in domain['street']]
domain['price_rent'] = [float(get_rent(i)) for i in domain['price']]
domain['coordinate'] = domain[['longitude','latitude']].values.tolist()

In [535]:
domain = domain[(domain.carpark_flag != 1) &
               (domain.price_rent > 40)]


In [672]:
import openrouteservice
domain_test = domain[domain['price_rent']<2000]
#domain_test = spark.createDataFrame(domain_test) 

In [679]:
#sample input [[13.42731, 52.51088], [13.384116, 52.533558]]
# return dictionary of distance (m) and duration (s)
def foot_dict(two_coordinates_list):
    
    route = client.directions(
    coordinates=two_coordinates_list,
    profile='foot-walking',
    format='geojson',
    options={"avoid_features": ["steps"]},
    validate=False,
)
    distance_duration = route['features'][0]['properties']['summary']
    return distance_duration


def car_dict(two_coordinates_list):
    
    route = client.directions(
    coordinates=two_coordinates_list,
    profile='driving-car',
    format='geojson',
    validate=False,
)
    distance_duration = route['features'][0]['properties']['summary']
    return distance_duration





In [680]:
#get centroid for CBD (postcode 3000)

postcode_df = gpd.read_file("../data/raw/POSTCODE/POSTCODE_POLYGON.shp")
postcode_df['centroid'] = postcode_df['geometry'].centroid
CBD_centroid = postcode_df.loc[postcode_df['POSTCODE'] == '3000']['centroid']
CBD_centroid =[CBD_centroid.values.x[0],CBD_centroid.values.y[0]]

/var/folders/7v/v1gh4ksn641cv86zx_w_yd840000gp/T/ipykernel_61531/3385975450.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  postcode_df['centroid'] = postcode_df['geometry'].centroid


In [700]:
domain_test['coordinate'][:5]

0    [144.964584, -37.8106079]
1    [144.956924, -37.8107758]
2    [144.952087, -37.8137741]
3    [144.952087, -37.8137741]
4    [144.960846, -37.8114166]
Name: coordinate, dtype: object

In [711]:
def car_CBD_dict(coordinates_list):
    dic = {'distance':[],'duration':[]}
    for i in coordinates_list:
        route = client.directions(
        coordinates= [i, CBD_centroid],
        profile='driving-car',
        format='geojson',
        validate=False,
    )
        dic['distance'].append(route['features'][0]['properties']['summary']['distance'])
        dic['duration'].append(route['features'][0]['properties']['summary']['duration'])
    return dic

In [ ]:
car_CBD_info = car_CBD_dict(domain_test['coordinate'][:])